### Effective Pandas Book Runthrough

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', 30)
pd.set_option("display.max_colwidth",3000)

In [ ]:
# pandas df with duckdb
import duckdb as db

con = db.connect(':memory:')
parquet_dir = r"E:/app_data/sec_apps_data/speed_test/filings_13f_full/filings_parquet"

%time df = db.from_parquet(f"{parquet_dir}/811360*.parquet", connection=con).df()

In [ ]:
%time df.head(2)

In [ ]:
# pandas df with vaex

import vaex
cikmanager = 811360
%time df1 = vaex.open(f"{parquet_dir}/{cikmanager}*.parquet")

In [ ]:
df1.head(2)

### Series

In [ ]:
series = {
'index ':[0, 1, 2, 3],
'data ':[145, 142, 38, 13],
'name':'songs'
}

In [ ]:
import pandas as pd

songs2 = pd.Series([145, 142, 38, 13], name='counts')
songs2
songs2.index

In [ ]:
import numpy as np

nan_series = pd.Series([2, np.nan],
index=['Ono', 'Clapton '])

nan_series
nan_series.count()
nan_series.size

In [ ]:
nan_series2 = pd.Series([2, None],
                        index=['Ono', 'Clapton '],
                        dtype='Int64')
nan_series2
nan_series2.count()

In [ ]:
nan_series2.astype('Int64')

In [ ]:
numpy_ser  = np.array([145, 142, 38, 13])
numpy_ser[1]

In [ ]:
songs3 = pd.Series([145, 142, 38, 13],
name='counts ',
index=['Paul', 'John', 'George ', 'Ringo '])

songs3

In [ ]:
# not filtering, but identifying rows as per condition
songs3 > songs3.median()

In [ ]:
# to filter rows based on rows values in certain columns
songs3[songs3 > songs3.median()]

In [ ]:
songs3.mean()
numpy_ser.mean()

In [ ]:
mask = songs3 > songs3.mean() # boolean array
mask
songs3[mask]

In [ ]:
# numpy equivalent of a mask
numpy_ser[numpy_ser > np.median(numpy_ser)]

In [ ]:
# categories
s = pd.Series(['m', 'l', 'xs', 's', 'xl'], dtype='category')

In [ ]:
s
s.cat.ordered

In [ ]:
# ordering it
s2 = pd.Series(['m', 'l', 'xs', 's', 'xl'])
size_type = pd.api.types.CategoricalDtype(categories=[ 's', 'm', 'l'], ordered=True)
s3 = s2.astype(size_type)
s3

In [ ]:
s3 > 's'

In [ ]:
s.cat.reorder_categories(['xs','s','m','l', 'xl'],
ordered=True)

In [ ]:
s3.str.upper()
s.str.upper()

In [ ]:
s_temp = pd.Series([32, 29, 20, 26, 31, 25, 30], dtype='int64')
s_temp
s_temp.mean()

In [ ]:
s_temp[s_temp > s_temp.mean()]

In [ ]:
s_colours = pd.Series(['blue', 'red', 'green', 'yellow', 'orange', 'purple', 'pink'], dtype='category')
s_colours

#### Series Deep Dive

In [ ]:
url = 'https://github.com/mattharrison/datasets/raw/master/data/' \
 'vehicles.csv.zip'
df = pd.read_csv(url)
city_mpg = df.city08
highway_mpg = df.highway08

In [ ]:
city_mpg

In [ ]:
len(dir(city_mpg))
dir(city_mpg)

In [ ]:
(city_mpg + highway_mpg) / 2

In [ ]:
s1 = pd.Series([10, 20, 30], index=[1,2,2])
s2 = pd.Series([35, 44, 53], index=[2,2,4], name='s2')

In [ ]:
s1.add(s2, fill_value=0)

In [ ]:
((city_mpg + highway_mpg) / 2) 

In [ ]:
# using chaining
(city_mpg
    .add(highway_mpg)
    .div(2))

In [ ]:
# mean
city_mpg.mean()

In [ ]:
city_mpg.is_unique

In [ ]:
city_mpg.is_monotonic_increasing

In [ ]:
f = pd.Series([1, 2, 3, 4 ])
f.is_unique

In [ ]:
f.is_monotonic_increasing

In [ ]:
city_mpg.quantile()

In [ ]:
city_mpg.quantile([0.25, 0.5, 0.75])

In [ ]:
city_mpg.quantile(0.8)

### Polars

In [ ]:
%%time
import polars as pl

df = pl.read_csv("https://j.mp/iriscsv")
print(df.filter(pl.col("sepal_length") > 5)
      .groupby("species")
      .agg(pl.all().sum())
)


In [ ]:
%%time
# lazy way
import polars as pl

print(
    pl.read_csv("https://j.mp/iriscsv")
    .lazy()
    .filter(pl.col("sepal_length") > 5)
    .groupby("species")
    .agg(pl.all().sum())
    .collect()
)


In [ ]:
%%time
# making df lazy
# filtering the df based on one column
(
    df.lazy()
    .filter(pl.col("sepal_length") > 5)
    .groupby("species")
    .agg(pl.all().sum())
    .collect()
)


In [ ]:

df.select([
    
    pl.col("species").sort().head(2),
    pl.col("sepal_length").filter(pl.col("sepal_length") == 5).sum()
])


In [ ]:

import polars as pl
import numpy as np

np.random.seed(12)

df = pl.DataFrame(
    {
        "nrs": [1, 2, 3, None, 5],
        "names": ["foo", "ham", "spam", "egg", None],
        "random": np.random.rand(5),
        "groups": ["A", "A", "B", "C", "B"],
    }
)
print(df)


In [ ]:

# unique values
out = df.select(
    [
        pl.col("names").n_unique().alias("unique_names_1"),
        pl.col("names").unique().count().alias("unique_names_2"),
    ]
)
print(out)


In [ ]:

out = df.select(
    [
        pl.sum("random").alias("sum"),
        pl.min("random").alias("min"),
        pl.max("random").alias("max"),
        pl.col("random").max().alias("other_max"),
        pl.std("random").alias("std dev"),
        pl.var("random").alias("variance"),
        pl.last("random").alias("last_var"),
        pl.first("random").alias("first_var"),
    ]
)
print(out)


In [ ]:
out = df.select(
    [
        pl.col("names").filter(pl.col("names").str.contains(r"o$")).count(),
    ]
)
print(out)


### Polars with my data

In [ ]:

import polars as pl
parquet_dir = r"E:\app_data\sec_apps_data\speed_test\filings_13f_full\filings_parquet"


df_filings = (pl.scan_parquet(f"{parquet_dir}/949509-*.parquet")
              .filter((pl.col("cikManager") == 949509) & (pl.col("filedAsOfDate") >= "2014-01-01" ) 
                      #& (pl.col("cusip") == "01609W102" ))
              ))
 
    # .filter(pl.col("sepal_length") > 5)
    # .groupby("species")
    # .agg(pl.all().sum())
    #.collect()
    

In [ ]:
df_filings.fetch(3)

In [ ]:
df_berk_2014up = df_filings.collect()
df_berk_2014up.head(4)

In [ ]:
df = (df_berk_2014up.filter(pl.col("report_Year") == 2014)
                  .groupby(["cikManager", "report_Year", "report_Quarter" ])
                #   .agg(pl.col("sharesHeldAtEndOfQtr").sum())
                  .agg(pl.col("sharesHeldAtEndOfQtr").last()) # 
                  )

In [ ]:
df

In [ ]:
# df = (df_berk_2014up.filter(pl.col("report_Year") == 2014)
#                   .groupby(["cikManager","report_Year" ])
#                 #   .agg(pl.col("sharesHeldAtEndOfQtr").sum())
#                   .agg(pl.col("sharesHeldAtEndOfQtr").sum()) # 
#                   )
# df

In [ ]:
## lazy mode
(df.lazy()
    .filter(pl.col("report_Quarter") != 1)
    .groupby("report_Quarter")
    .agg(pl.all().sum())
    .collect()
)

In [ ]:
## Expressions
pl.col("foo").sort(reverse=True).head(2)

df.select([pl.col("report_Quarter").sort(reverse=True).head(1),
           pl.col("sharesHeldAtEndOfQtr").filter(pl.col("report_Quarter") == 3).min()
           
           
])

In [ ]:
## unique values
out = df_filings.select([
    pl.col("cusip").n_unique().alias("unique_cusip_1"),
    pl.col("cusip").unique().count().alias("unique_cusip_2"),
                         
                         
                         ])
print(out.collect())

In [ ]:
## aggregations

out = df_filings.select([
    pl.sum("sharesHeldAtEndOfQtr").alias("shares_sum"),
    pl.min("sharesHeldAtEndOfQtr").alias("shares_min"),
    pl.max("sharesHeldAtEndOfQtr").alias("shares_max"),
    pl.col("sharesHeldAtEndOfQtr").max().alias("shares_other_max"),
    pl.std("sharesHeldAtEndOfQtr").alias("shares_std_dev"),
    pl.var("sharesHeldAtEndOfQtr").alias("variance"),
    pl.col("report_Quarter").alias("quarter"),
    pl.col("sharesHeldAtEndOfQtr").alias("shares"),
    pl.col("sharesHeldAtEndOfQtr").shift(-1).alias("shares_prev"),
    pl.col("cusip").str.to_uppercase().alias("cusip")
                         
                         
                         
                         ])
print(out.collect())


In [ ]:
pl.Config.set_tbl_rows(50)
pl.Config.set_tbl_cols(50)
pl.Config.set_tbl_width_chars(50)
pl.Config.set_fmt_str_lengths(50)

(df_filings 
    .sort(["cikManager", "cusip", "periodOfReport","filedAsOfDate", "accessionNumber"], reverse=True)
    .select([
      
    pl.col("cusip"),
    (pl.col("periodOfReport").dt.year()+"Q"+pl.col("periodOfReport").dt.quarter()).alias("quarter"),
    pl.col("periodOfReport"),
    #pl.col("filedAsOfDate").alias("fdate"),
    pl.col("sharesHeldAtEndOfQtr").first().over(["cikManager", "cusip"]).alias("shares"),
    # pl.col("sharesHeldAtEndOfQtr").sum().over(["cikManager", "cusip", "periodOfReport"]).alias("shares_sum"),
    pl.col("sharesHeldAtEndOfQtr").list().over(["cusip", "periodOfReport"]).alias("shares_list")
    ])
).collect()

In [ ]:
(df_filings
    .groupby(["cusip", (pl.col("periodOfReport").dt.year()+"Q"+pl.col("periodOfReport").dt.quarter()).alias("quarter")])
    .agg([pl.col("sharesHeldAtEndOfQtr").first(),
          #pl.col("cusip").unique(),
          pl.col("filedAsOfDate").first(),
          pl.col("accessionNumber").first()
          ])
    .sort(["cusip", "quarter","filedAsOfDate", "accessionNumber"], reverse=True)
    ).collect()

In [ ]:
bad_cusips = ["000000000", "", "0"]
# (pl.col("cusip") != "000000000") & (pl.col("cusip") != "") & (pl.col("cusip") != "0")
(df_filings
    .filter((pl.col("cusip").is_in(bad_cusips).is_not())) # filter out bad cusips
    .groupby([pl.col("periodOfReport").dt.year()])
    .agg([
        pl.col("accessionNumber").first(),
        pl.col("accessionNumber").str.slice(-6).cast(pl.Int32).first().alias("new_accession_number"),
        pl.count().alias("count"),
        pl.col("submissionType"),
        pl.first("cusip")       
    ])
    
    .sort("accessionNumber", reverse=True)
    .limit(5)
).collect()

In [ ]:

(df_filings
    .filter((pl.col("cusip").is_in(bad_cusips).is_not())) # filter out bad cusips
    .sort(["cusip", "periodOfReport", "filedAsOfDate", "accessionNumber"], reverse=True)
    .groupby(["cusip", (pl.col("periodOfReport").dt.year()+"Q"+pl.col("periodOfReport").dt.quarter()).alias("quarter")])
    .agg([
        pl.col("periodOfReport").first().alias("periodOfReport"),
        pl.count().alias("count"),
        pl.col("ID").count().alias("ID_count"),
        pl.col("filedAsOfDate").list().alias("list_of_fdates"),
        pl.col("sharesHeldAtEndOfQtr").list().alias("list_of_shares"),
        pl.col("sharesHeldAtEndOfQtr").first().alias("new_accession_number"),
        
        pl.col("filedAsOfDate").first().alias("filedAsOfDate")
        
       
    ])
    
    .sort(["periodOfReport", "filedAsOfDate"], reverse=True)
    #.limit(1)
).collect()

In [ ]:
## Conditional aggregations
import polars as pl

bad_cusips = ["000000000", "", "0"]
(df_filings
    .filter((pl.col("cusip").is_in(bad_cusips).is_not())) # filter out bad cusips
    .sort(["cusip", "periodOfReport", "filedAsOfDate", "accessionNumber"], reverse=True)
    .groupby(["cusip", (pl.col("periodOfReport").dt.year()+"Q"+pl.col("periodOfReport").dt.quarter()).alias("quarter")])
    .agg([
        pl.col("periodOfReport").first().alias("periodOfReport"),
        
        pl.col("ID").count().alias("ID_count"),
        pl.col("filedAsOfDate").list().alias("list_of_fdates"),
        pl.col("sharesHeldAtEndOfQtr").list().alias("list_of_shares"),
        pl.col("sharesHeldAtEndOfQtr").first().alias("new_accession_number"),
        pl.col("filedAsOfDate").first().alias("filedAsOfDate"),
        pl.col("accessionNumber").str.slice(-6).cast(pl.Int32).first().alias("accession_rank"),
        (pl.col("submissionType") == "13F-HR").sum().alias("original"),
        (pl.col("submissionType") == "13F-HR/A").sum().alias("amendement")
        
       
    ])
    
    .sort(["periodOfReport", "filedAsOfDate"], reverse=True)
    #.limit(1)
).collect()

In [ ]:
import polars as pl

bad_cusips = ["000000000", "", "0"]
(df_filings
    .filter((pl.col("cusip").is_in(bad_cusips).is_not())) # filter out bad cusips
    .sort(["cusip", "periodOfReport", "filedAsOfDate", "accessionNumber"], reverse=True)
    .groupby(["cusip", "submissionType"])
    .agg([(pl.col("submissionType").count()).alias("submission_count")
    .filter((pl.col("submissionType") == "13F-HR") | (pl.col("submissionType") == "13F-HR/A"))
    
      
       
    ])
    .sort("submission_count", reverse=True)
    
    
    #.limit(1)
).collect()

In [ ]:
## Apply filtering within a group and use a custom function
import polars as pl

def filing_age() -> pl.Expr:
    # return pl.col("filedAsOfDate").cast(pl.DateType()) - pl.col("periodOfReport").cast(pl.DateType())
    return pl.col("filedAsOfDate") - pl.col("periodOfReport")

def avg_count_submission_type(type: str) -> pl.Expr:
    return filing_age().filter(pl.col("submissionType") == type).mean().alias(f"Average filing age for {type}")



bad_cusips = ["000000000", "", "0"]
(df_filings
    .groupby(["cusip", "periodOfReport"])
    .agg([
    avg_count_submission_type("13F-HR"),
    avg_count_submission_type("13F-HR/A"),
    (pl.col("submissionType") == "13F-HR").sum().alias("original"),
    (pl.col("submissionType") == "13F-HR/A").sum().alias("amendement"),
       
    ])
    
    #.limit(1)
).collect()

In [ ]:
## sorting and then grouping aaaand dorting again within a group
# we can also sort by another column in a group (still needs mental clarification )



import polars as pl

def get_cusip_filedat() -> pl.Expr:
    return pl.lit("Filed on..") + pl.col("filedAsOfDate")

bad_cusips = ["000000000", "", "0"]
(df_filings
    .sort(["cusip","periodOfReport", "filedAsOfDate"], reverse=True)
    .groupby(["cusip", "periodOfReport"])
    .agg([
        get_cusip_filedat().first().alias("last filing"),
        get_cusip_filedat().last().alias("first filing"),
        get_cusip_filedat().sort().last().alias("alphabetical filing"),
        pl.col("submissionType").sort_by("filedAsOfDate").first().alias("first filing type"),
        
          
    ])
    .limit(5)
    
    ).collect()

In [ ]:
# fold
out = df_filings.select(
    pl.fold(acc=pl.lit(0), f=lambda acc, x: acc + x, exprs=pl.col("*")).alias("sum"),
)
out.collect()



In [ ]:
## not sure how this works

out = df_filings.filter(
    pl.fold(acc=pl.lit(True), f=lambda acc, x: acc & x, exprs=pl.col("report_Quarter") >1),
)
out.collect()

In [ ]:
## fold and concat_str

out = df_filings.select([
        pl.concat_str(["report_Year", pl.lit("Q"), "report_Quarter"])
    
                 ])
out.collect()

In [ ]:
## split the string and get the necessary element of the list

out = df_filings.select([
        pl.concat_str(["report_Year", pl.lit("Q"), "report_Quarter"]),
        pl.col("accessionNumber").str.split(by="-").arr.get(-1).cast(pl.UInt32)
    
                 ])
out.collect()

In [ ]:
## window functions

from aioitertools import groupby


out = df_filings.select([
    "report_Year",
    "report_Quarter",
    "sharesHeldAtEndOfQtr",
    pl.col("valueTotal").mean().over("report_Year").alias("mean_valueTotal_by_year"),
    pl.col("sharesHeldAtEndOfQtr").mean().over(["report_Year"]).alias("mean_shares_by_year"),
    pl.col("sharesHeldAtEndOfQtr").mean().over(["report_Year", "report_Quarter"]).alias("mean_shares_by_year_and_quarter"),
    pl.col("sharesHeldAtEndOfQtr").mean().alias("mean_shares")
    
    
    
])
(out
.groupby(["report_Year", "report_Quarter"]).agg([pl.col("sharesHeldAtEndOfQtr").first()])
.sort(["report_Year", "report_Quarter"], reverse=True)).collect()


In [ ]:
## filter and select columns

filtered = \
df_filings.filter(pl.col("cusip") != "").select([
    "report_Year",
    "cusip",
    "sharesHeldAtEndOfQtr",
    
])

filtered.fetch(3)


# let's sort it by cusip and then shares
out = filtered.with_columns([
    pl.col(["cusip", "sharesHeldAtEndOfQtr"]).sort(reverse=True).over("cusip")
    ])  
out.collect()

In [ ]:
# sort and select

# the different groppings by is confusing for fe for now
out = df_filings.sort(["cusip", "periodOfReport"]).select([
    pl.col("cusip").head(3).list().over("cusip").flatten(),
    pl.col("periodOfReport").sort_by(pl.col("sharesHeldAtEndOfQtr"), reverse=True).head(3).list().over("cusip").flatten().alias("fastest/group")
    #pl.col("")
    
    
])
out.collect()

# from the guide
# out = df.sort("Type 1").select(
#     [
#         pl.col("Type 1").head(3).list().over("Type 1").flatten(),
#         pl.col("Name").sort_by(pl.col("Speed")).head(3).list().over("Type 1").flatten().alias("fastest/group"),
#         pl.col("Name").sort_by(pl.col("Attack")).head(3).list().over("Type 1").flatten().alias("strongest/group"),
#         pl.col("Name").sort().head(3).list().over("Type 1").flatten().alias("sorted_by_alphabet"),
#     ]
# )

In [ ]:



df_filings.select([
    (pl.col("submissionType") + pl.col("cikManager")).alias("original"),
    (pl.col("sharesValue") + pl.col("valueTotal")).alias("sum")
    
    
]).collect()

In [ ]:

from IPython.display import display
import polars as pl
pl.Config.set_tbl_rows(20)
pl.Config.set_tbl_width_chars(50)
pl.Config.set_fmt_str_lengths(50)
#display(out.collect())

In [ ]:
import polars as pl
parquet_dir = r"E:\app_data\sec_apps_data\speed_test\filings_13f_full\filings_parquet"


df_filings = (pl.scan_parquet(f"{parquet_dir}/949509-*.parquet")
              .filter((pl.col("cikManager") == 949509) & (pl.col("filedAsOfDate") >= "2014-01-01" ) 
                      #& (pl.col("cusip") == "01609W102" ))
              ))
 
    # .filter(pl.col("sepal_length") > 5)
    # .groupby("species")
    # .agg(pl.all().sum())
    #.collect()

In [ ]:
df_filings.collect().head(3)

In [ ]:
# out = grades.select([
#     pl.concat_list(pl.all().exclude("student")).alias("all_grades")
# ])

# concatenating to list
out = df_filings.select([pl.concat_list(["valueTotal", "sharesValue"]).alias("sum")])
out.fetch(3)


In [ ]:
rank_pct = pl.element().rank(reverse=True) / pl.col("").count()

df_filings.with_column(
    pl.concat_list(["valueTotal", "sharesValue"]).alias("values")).select([
        pl.all().exclude("values"),
        pl.col("values").arr.eval(rank_pct, parallel=True).alias("values_rank")
    ]).collect().head(3)
    


### Pandas Course - Pluralsite - B3. Cleaning Data. Python Data Playbook (Chris Achard, 2018)

In [ ]:
import pandas as pd


In [ ]:
data = pd.read_csv(r"course/data/exercise/artwork_data.csv")
data.head(3)

In [ ]:
data.dtypes

In [ ]:

data.acquisitionYear.astype(float)

In [ ]:
data.acquisitionYear.dtype

data.acquisitionYear = data.acquisitionYear.astype(float)
data.acquisitionYear.dtype

In [ ]:
fulldf = pd.read_csv(r"course/data/exercise/artwork_data.csv", low_memory=False)
fulldf.head(3)

In [ ]:
from numpy import full


fulldf.dtypes
# fulldf.height.astype(float)

# pd.to_numeric(fulldf.height)
fulldf.height[41339]

pd.to_numeric(fulldf.height, errors="coerce")
pd.to_numeric(fulldf.height, errors="coerce")[41339] # it turened the 'mm' to nan

fulldf.height.dtype # still an object, we need to convert to float
fulldf.height = pd.to_numeric(fulldf.height, errors="coerce")

fulldf.height.dtype # now it's a float
 

 

#### Aggregation

In [ ]:
data = pd.read_csv(r"course/data/exercise/artwork_sample.csv", low_memory=False)
data.head(3)

In [ ]:
data.year = pd.to_numeric(data.year, errors="coerce")

In [ ]:
data.dtypes,
data.columns

In [ ]:
data.year.min()

In [ ]:
data.year.max()

In [ ]:
data.year.sum()

In [ ]:
data.year.mean()

In [ ]:
data.year.std()

In [ ]:
data.artist.min() # min() could be applied to a string too, but mean() would not make sense

In [ ]:
# if to sum() string values, it will just concatenate them
data.artist.sum()


In [ ]:
data.sum() # not much sense in doing it, but we can use sum() on the dataframe

In [ ]:
data.agg("min") # this would get the min() of each column

In [ ]:
data.agg(['min', 'max']) # this would get the min() and max() of each column

In [ ]:
data.agg(['min', 'max', 'mean', 'std'])

In [ ]:
# flipping the axis to get the mean of each column and not a row
data.agg({'id' : ['sum', 'min'], 'depth' : ['sum', 'max']}) # this would get the mean of each numerical column


In [ ]:
### example of my data and groupby
# pandas reading
import glob
parquet_dir = r"E:/app_data/sec_apps_data/speed_test/filings_13f_full/filings_parquet/"
import pandas as pd
pandas_df = pd.read_parquet(glob.glob(f"{parquet_dir}949509-*.parquet"))
pandas_df.sort_values(by=['cusip', 'periodOfReport', 'filedAsOfDate'],ascending=False) \
#groupby(['cusip', 'periodOfReport', 'filedAsOfDate'])['sharesHeldAtEndOfQtr'].agg(pd.Series.max).tail(10)

In [ ]:
data = pd.read_csv(r"course/data/exercise/artwork_sample.csv", low_memory=False)
data.head()

In [ ]:
data.dtypes

In [ ]:
data.height.mean()
data.height

In [ ]:
# standardize the height
height = data.height


In [ ]:
norm = (height - height.mean()) / height.std()
norm

In [ ]:
# normalize the height so it's between 0 and 1
minmax = (height - height.min()) / (height.max() - height.min())
minmax

In [ ]:
minmax.min(), minmax.max()

In [ ]:
data.height = minmax
data.head()

In [ ]:
data['standardized_height'] = norm
data

In [ ]:
## Transforming data
import pandas as pd
data = pd.read_csv(r"course/data/exercise/artwork_sample.csv", low_memory=False)
data.head()


In [ ]:
data.height.transform(lambda x: print(  x))

In [ ]:
data.height.transform(lambda x: x)

In [ ]:
data.height.transform(lambda x: x/10)

In [ ]:

 # transform on groupby
 data.groupby('artist')

In [ ]:

data.groupby('artist').transform('nunique')

In [ ]:

data.artist.nunique()
# data.groupby(['artist', 'medium']).transform('nunique')

data.groupby('artist')['height'].transform('mean')

In [ ]:
data.artist

In [ ]:
data['mean_height_by_artist'] = data.groupby('artist')['height'].transform('mean')

In [ ]:
data

In [ ]:
# Filter data

import pandas as pd
data = pd.read_csv(r"course/data/exercise/artwork_data.csv", low_memory=False)
data.head()

In [ ]:
# reducing the dataframe to only the columns we need
data.filter(items=['id','artist', 'artistId'])

In [ ]:
# if I don't know the exact columns name
data.filter(like='year')

In [ ]:
# case insensitive regex
data.filter(regex="(?i)year")

In [ ]:
# we can use filter on rows too by using axis=0
data.filter(axis=0, regex="^100.$") # only fiters on index's values

In [ ]:
 # only even numbers of index
data.filter(axis=0, regex="^100(0|2|4|6|8)$")

In [ ]:
## dropping columns
import pandas as pd

In [ ]:
data = pd.read_csv(r"course/data/exercise/artwork_sample.csv", low_memory=False)
data.head()

In [ ]:
data.drop(0)  #drop the first row with the index 0

In [ ]:
# drop a column
data.drop('id',axis=1)

In [ ]:
data.drop(columns=['id'])

In [ ]:
data.drop(columns=['id', 'height', 'width', 'depth']) 

In [ ]:
# to drop rows we pass a list of labels
data.drop(labels=[0, 1, 2])

In [ ]:
# to drop permanently we use inplace=True
data.drop(columns=['id'], inplace=True)

In [ ]:
data

In [ ]:
# to avoid working with the columns we don't need in the first place, we can drop them when reading the data in
data = pd.read_csv(r"course/data/exercise/artwork_sample.csv", low_memory=False, usecols=['artist', 'title'])
data


In [ ]:
# chaning the casing of the columns
data = pd.read_csv(r"course/data/exercise/artwork_sample.csv", low_memory=False)
data.columns



In [ ]:
data.columns.str.lower()

In [ ]:
#[x.lower() for x in data.columns]
data.columns = [x.lower() for x in data.columns]

In [ ]:
data.columns

In [ ]:
# with map()
data = pd.read_csv(r"course/data/exercise/artwork_sample.csv", low_memory=False)


In [ ]:
data.columns = map(lambda x: x.lower(), data.columns)

In [ ]:
data.columns

In [ ]:
data = pd.read_csv(r"course/data/exercise/artwork_sample.csv", low_memory=False)

In [ ]:
# with regex : we look for uppercase letters, and add '_' right before it and cast all to lowercase
import re
data.columns = [re.sub(r'([A-Z])', r'_\1', x).lower() for x in data.columns]

In [ ]:
data.columns

In [ ]:
# reanming columns
data = pd.read_csv(r"course/data/exercise/artwork_sample.csv", low_memory=False)
data.columns

In [ ]:
data.rename(columns={"thumbnailUrl": "thumbnail"})
data.rename(columns={"thumbnailUrl": "thumbnail"}, inplace=True)

In [ ]:
data.head()

In [ ]:
data.rename(columns=lambda x: x.lower(), inplace=True)

In [ ]:
data.columns

In [ ]:
data.columns = ['id', 'Accession_number', 'Artist', 'Artistrole', 'artistid', 'title',
       'datetext', 'medium', 'creditline', 'year', 'acquisitionyear',
       'dimensions', 'width', 'height', 'depth', 'units', 'inscription',
       'thumbnailcopyright', 'thumbnail', 'url']

In [ ]:
data.columns

In [ ]:
data = pd.read_csv(r"course/data/exercise/artwork_sample.csv", low_memory=False, names=\
    ['id', 'Accession_number', 'Artist', 'Artistrole', 'artistid', 'title',
       'datetext', 'medium', 'creditline', 'year', 'acquisitionyear',
       'dimensions', 'width', 'height', 'depth', 'units', 'inscription',
       'thumbnailcopyright', 'thumbnail', 'url'], header=0)
data.columns

### Indexing and Filtering

In [ ]:
import pandas as pd
data = pd.read_csv(r"course/data/exercise/artwork_sample.csv", low_memory=False)
data

In [ ]:
data['id'] # for one column

In [ ]:
data['id'][1] # 

In [ ]:
# to get multiple columns
data[['artist', 'title']]
# data.filter(items=['artist', 'title']) # same as above

In [ ]:
data[0:5] 

In [ ]:
data[['artist', 'title']][1:2]

In [ ]:
data[1:5]['artist']

In [ ]:
# filter on rows
data[data['year'] > 1800] # we filter on row value and select all df

In [ ]:
# if we want to filter on row and select only a subset of columns
data[data.year > 1800][['artist', 'title']]
data[data.year > 1800]['year']

### loc and iloc

In [ ]:
import pandas as pd
data = pd.read_csv(r"course/data/exercise/artwork_sample.csv", low_memory=False)
data

In [ ]:
# loc[rows, columns]
# loc uses labels to select rows and columns. In case of index, in our case it's the row number, but it can be anythin else
data.loc[0, :]

In [ ]:
data.loc[0:1, ['artist', 'title']]

data.loc[0:2, :]

In [ ]:
data.loc[0:2,'title']

In [ ]:
data.loc[0:2, ['artist', 'title']]

In [ ]:
# we can also use precise rows we need. Not the range of rows
data.loc[[1,5], ['artist', 'title']]

In [ ]:
# we can also slise columns
data.loc[0:1, 'id': 'title']

In [ ]:
# we can also filter with loc
data.loc[data.artist == 'Blake, Robert',:]
data.loc[data.artist == 'Blake, Robert', 'title']
data.loc[data.artist == 'Blake, Robert', 'title':'year']
data.loc[data.artist == 'Blake, Robert', ['artist','year']]

### iloc

In [ ]:
import pandas as pd
data = pd.read_csv(r"course/data/exercise/artwork_sample.csv", low_memory=False)
data

In [ ]:
# iloc uses integer positions of rows and columns
data.iloc[0,:]
data.iloc[0:3,]

In [ ]:
# to better see the difference it makes for the index, we can use a different index
data.set_index('id', inplace=True)
data

In [ ]:
data.iloc[0:3,]

In [ ]:
data.loc[1035:1037,:]

In [ ]:
# the same integer positin is used for columns
data.iloc[0:3,0:3]

In [ ]:
# if we reset the index inplace, all will go back to the original index
data.reset_index(inplace=True)

In [ ]:
data
data.iloc[0:3,0:3]

In [ ]:
# select spedific rows and columns and not the range
data.iloc[[5,8],[0,3, 7]]

### Filtering with str.contains

In [ ]:
import pandas as pd
data = pd.read_csv(r"course/data/exercise/artwork_sample.csv", low_memory=False)
data

In [ ]:
data.medium.str.contains('Graphite')

In [ ]:
# we can pass this to .loc
data.loc[data.medium.str.contains('Graphite'), ['artist','title', 'medium']] # default ir case sensitive

In [ ]:
data.loc[data.medium.str.contains('Graphite', case=False), ['artist','title', 'medium']] # case insensitive

In [ ]:
# we can do the same with regex
data.loc[data.medium.str.contains('(?i)Graphite', regex=True), ['artist','title', 'medium']]

In [ ]:
# multiple filters with regex
data.loc[data.medium.str.contains('(?i)Graphite', regex=True) \
         | data.medium.str.contains('(?i)Line', regex=True), ['artist','title', 'medium']] 

In [ ]:
# a simpler version
data.loc[data.medium.str.contains('graphite|line', case=False, regex=True), ['artist','title', 'medium']]

In [ ]:
# to use a str method on not a string, we first need to convert it to a string
data.year.astype(str).str.contains('1826')

In [ ]:
# to filter
data.loc[data.year.astype(str).str.contains('1826')]

In [ ]:
data = pd.read_csv(r"course/data/exercise/artwork_data.csv", low_memory=False)
data

In [ ]:
data.dtypes

In [ ]:
# when working with strings or non strings converted to strings, if there are NaNs, we need actiate na=False
data.loc[data.dimensions.str.contains('support', na=False)]

### Cleaning Data

In [ ]:
import pandas as pd

data = pd.read_csv(r"course/data/exercise/artwork_data.csv", low_memory=False)
data

In [ ]:
# deal with whitespaces
# this identifies rows where in the column 'title' there are whitespaces at the end of the string
data.loc[data.title.str.contains('\s$', regex=True)]

In [ ]:
# this line deletes the whitespaces from the columln 'title'
data.title.str.strip()

In [ ]:
data.title = data.title.str.strip()

In [ ]:
data.loc[data.title.str.contains('\s$', regex=True)]

In [ ]:
data.title.str.lstrip() # left strip
data.title.str.rstrip() # right strip
# with transform
# a reminder - this only identifies rows, not deletes them
data.title.transform(lambda x: x.str.strip())


### Replace bad data with NaN

In [ ]:
import pandas as pd

data = pd.read_csv(r"course/data/exercise/artwork_data.csv", low_memory=False)
data

In [ ]:
# check if the df has nans
data.isna().any()
# data.isna().all() # if an entire row has nans

In [ ]:
# check how many nans in which columns
data.isna().sum()

In [ ]:
data.isna().mean().mul(100) # percentage of nans in each column

In [ ]:
# mask to identify the rows where there are NaNs for the column 'medium'
pd.isna(data.loc[:, 'medium'])

# and show the entire df but filter out only where there are nans in 'medium' 
data.loc[pd.isna(data.loc[:, 'medium'])]

In [ ]:
from numpy import nan


In [ ]:
data.replace({'dateText': {'date not known': nan}}, inplace=True)
data

In [ ]:

data = pd.read_csv(r"course/data/exercise/artwork_data.csv", low_memory=False)
data

In [ ]:
# another way to deal with selecting rows with what we see as bad data, then selecting only the needed column 
data.loc[data.dateText == 'date not known', ['dateText']]

In [ ]:
# and then# assigning NaN to the bad rows of that column
data.loc[data.dateText == 'date not known', ['dateText']] = nan

In [ ]:
# select rows where the year is NOT nan and also has a value other than a number
data.loc[data.year.notnull() & data.year.astype(str).str.contains('[^0-9]', regex=True)] 
# data.loc[data.year.astype(str).str.contains('[^0-9]', regex=True)]

In [ ]:
# be very careful with not forgetting to indicate which columns exactly needs to be set as NaN. If not, all df 
# will be set to NaN
data.loc[data.year.notnull() & data.year.astype(str).str.contains('[^0-9]', regex=True), ['year']] = nan

In [ ]:
data

In [ ]:
data.iloc[67968:67969]

### Filling missing data with value


In [ ]:
data = pd.read_csv(r"course/data/exercise/artwork_data.csv", low_memory=False)
data

In [ ]:
data.fillna(0)
data.depth.fillna(0)
data.fillna(value={'depth':0})
data.fillna(value={'depth':0}, inplace=True)

In [ ]:
data

### Dropping rows of data

In [ ]:
data = pd.read_csv(r"course/data/exercise/artwork_data.csv", low_memory=False)
data.shape

In [ ]:
# to drop rows with nan in the entire df, if a nan is in any column
data.dropna()

In [ ]:
# it leaves only rows without nans
data.dropna(how='any').shape

In [ ]:
# data.dropna() is the same as data.dropna(how='any')
 # dropna(how='all') drops rows where all the values in all the columns in a row are NaN
data.dropna(how='all').shape # since the shape is the same as the original one, it means there are no rows with all nans

In [ ]:
data.dropna(thresh=14).shape # thresh sets up how many columns must have a NaN in the dame row for it to be dropped

In [ ]:
# to only drop rows with NaN in specific columns
# data.dropna(subset=['year', 'acquisitionYear']).shape
# the same as with 'any'
# data.dropna(subset=['year', 'acquisitionYear'], how='any').shape # any of the columns must have a NaN
data.dropna(subset=['year', 'acquisitionYear'], how='all').shape # all of the columns must have a NaN


In [ ]:
data.shape

In [ ]:
# to actually change the df
data.dropna(subset=['year', 'acquisitionYear'], inplace=True)

In [ ]:
data.shape

### Duplicate values

In [ ]:
data = pd.read_csv(r"course/data/exercise/artwork_sample.csv", low_memory=False)
data

In [ ]:
data.drop_duplicates() # drops rows with all columns in a row being duplicates

In [ ]:
data.drop_duplicates(subset=['artist']) # drops rows with only some colulmns being duplicates

In [ ]:
data.drop_duplicates(subset=['artist'], keep='first') # we can specify which rows to keep if there are duplicates

In [ ]:
data.drop_duplicates(subset=['artist'], keep='last') # we can specify which rows to keep if there are duplicates

In [ ]:
data.drop_duplicates(subset=['artist'], keep=False) # or we can drop both duplicates

In [ ]:
data.drop_duplicates(subset=['artist'], keep='first', inplace=True) # we can specify which rows to keep if there are duplicates

In [ ]:
data

In [ ]:
# with full dataset
data = pd.read_csv(r"course/data/exercise/artwork_data.csv", low_memory=False)
data

In [ ]:
data.shape

In [ ]:
# to just see the duplicates
data.duplicated()  # returns only for fully duplicate rows

In [ ]:
data.loc[data.duplicated()]

In [ ]:
# show duplicats in specific columns
data.duplicated(subset=['artist', 'title'], keep=False) # this is crucial to identify the duplicates. 
# We need to set the keep to False, so the duplicates are highlighted as True



In [ ]:
data.loc[data.duplicated(subset=['artist', 'title'], keep=False) ]

In [ ]:
# findd a row with certain value in a column
data.loc[data.title.str.contains('The Circle of the Lustful: Francesca da ')]

### Another Course on pandas


In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

df = pd.

(r"E:/Downloads/_Pandas/Pandas Playbook. Manipulating Data (Reindert-Jan Ekker, 2018)/weather.csv")
df

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.head(50)

In [ ]:
df.describe()

In [ ]:
df.mean()

In [ ]:
df.max()

In [ ]:
df['PRESSURE'].mean()

In [ ]:
# find out the most common temperature
df.TEMP.mode()

In [ ]:
# all the values and their counts for a column
df['TEMP'].value_counts()

In [ ]:
# some plotting
df['TEMP'].plot()

In [ ]:
df['TEMP'].plot.hist()

In [ ]:
df['TEMP'].plot.hist(bins=50)

### Indexing

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv(r"E:/Downloads/_Pandas/Pandas Playbook. Manipulating Data (Reindert-Jan Ekker, 2018)/weather.csv").head()
df

In [ ]:
df['TEMP']

In [ ]:
 df['TEMP'][1]

In [ ]:
dft = df.T
dft

In [ ]:
dft.columns

In [ ]:
dft[2]

In [ ]:
dft[2]['TIME']

In [ ]:
dft[2][2]

In [ ]:
t = pd.DataFrame([['John'], ['Bob'], ['Anne']], index=[4, 3, 4])
t

In [ ]:
t[0][4]

In [ ]:
df[['PRESSURE', 'TIME', 'TEMP']]

In [ ]:
df['TIME'][[3, 1, 4]]

In [ ]:
df[2:4]

In [ ]:
df[2:4][['TEMP', 'PRESSURE']]

In [ ]:
df[['TEMP', 'PRESSURE']][2:4] # slices are only used for rows

In [ ]:
dft[3:][[1, 3]]

In [ ]:
df['PRESSURE'][2:4]

In [ ]:
dft

In [ ]:
dft[:2] # with integer clicing, the end is not included, but with string slicing, the end IS included

In [ ]:
dft['TIME':'PRESSURE']

### loc and iloc

In [ ]:
capitals = pd.DataFrame(
    [
    ["Ngerulmud",391,1.87],
    ["Vatican City",826,100],
    ["Yaren",1100,10.91],
    ["Funafuti",4492,45.48],
    ["City of San Marino",4493]
    ], 
    index = ["Palau", "Vatican City", "Nauru", "Tuvalu", "San Marino"],
    columns=['Capital', 'Population', 'Percentage'])

In [ ]:
capitals

In [ ]:

capitals.loc['Nauru', 'Population']

In [ ]:
# the same as above
capitals['Population']['Nauru']

In [ ]:
capitals.loc[['Nauru', 'Tuvalu'], ['Population', 'Percentage']]

In [ ]:
capitals.loc['Palau': 'Nauru', ['Population', 'Percentage']]

In [ ]:
capitals.loc[['San Marino', 'Vatican City']]

In [ ]:
capitals.iloc[[4, 1]]

In [ ]:
capitals.iloc[[4, 1], 1:]

In [ ]:
capitals.iloc[:, 2]

### Filtering

In [ ]:
capitals

In [ ]:
# filtering with boolean indexing
capitals[[True, True, False, True, False]]

In [ ]:
capitals['Percentage'] > 25

In [ ]:
capitals[capitals['Percentage'] > 25]

In [ ]:
grades = pd.DataFrame([[6, 4], [7, 8], [6, 7], [6, 5], [5, 2]], 
                       index = ['Mary', 'John', 'Ann', 'Pete', 'Laura'],
                       columns = ['test_1', 'test_2'])
grades

In [ ]:
grades['test_2'] <= grades['test_1']

In [ ]:

grades[grades['test_2'] <= grades['test_1']]

In [ ]:
grades.mean()


In [ ]:
grades.mean() > 5.5

In [ ]:
grades.loc[:, grades.mean() > 5.5]

### Changing data

In [ ]:
grades

In [ ]:
grades.loc[['Laura', 'John'], 'test_2'] += 1
# the same as above
grades.loc[['Laura', 'John'], 'test_2'] = grades.loc[['Laura', 'John'], 'test_2'] + 1
grades

In [ ]:
grades['test_1'] += .5
grades

In [ ]:
grades.loc['Mary'] += 2
grades

In [ ]:
grades.loc['Pete'] = [7, 9]
grades

In [ ]:
# wrong way to do it as the seconds command will work on strings and not on numbers
 
grades[grades < 6] = 'Fail'
grades[grades >= 6] = 'Pass'
grades

In [ ]:
grades = pd.DataFrame([[6, 4], [7, 8], [6, 7], [6, 5], [5, 2]], 
                       index = ['Mary', 'John', 'Ann', 'Pete', 'Laura'],
                       columns = ['test_1', 'test_2'])

In [ ]:
# right way to do it
failing = grades < 6
passing = grades >= 6
grades[failing] = 'Fail'
grades[passing] = 'Pass'
grades

In [ ]:
grades = pd.DataFrame([[6, 4], [7, 8], [6, 7], [6, 5], [5, 2]], 
                       index = ['Mary', 'John', 'Ann', 'Pete', 'Laura'],
                       columns = ['test_1', 'test_2'])

In [ ]:
# average of the grades by rows
grades.mean(axis=1)

In [ ]:
grades.mean(axis=1) > 6 # boolean indexing

In [ ]:
# adding a new column
grades['passed'] = grades.mean(axis=1) > 6
grades

In [ ]:
# assigning new values
# do not use.... grades['test_2]["Ann"] = 7

# correct way is with loc
grades.loc['Ann', 'test_2'] = 8
grades

### Sortings

In [ ]:
capitals.sort_index(ascending=False)

In [ ]:
capitals.sort_index(ascending=False, inplace=True)
capitals

In [ ]:
capitals.sort_index(axis=1)
capitals

In [ ]:
capitals.sort_values(by='Percentage')

In [ ]:
grades.sort_values(by=['test_1', 'test_2'])

In [ ]:
grades.sort_values(by=['test_1', 'test_2'], ascending=False)

In [ ]:
# specify the ascending or descending order for each column
grades.sort_values(by=['test_1', 'test_2'], ascending=[True, False], inplace=True)
grades


### pandas through with duckdb

In [ ]:
import duckdb as db
con = db.connect(':memory:')
dir = r"E:/Downloads/_Pandas/Pandas Playbook. Manipulating Data (Reindert-Jan Ekker, 2018)/weather.csv"
df = con.execute(f"SELECT * FROM read_csv_auto('{dir}')").df()



In [ ]:
df.sort_values(by=['MONTH', 'DAY'], ascending=[True, False])

### Missing Data

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv(r"E:/Downloads/_Pandas/Pandas Playbook. Manipulating Data (Reindert-Jan Ekker, 2018)/weather_m4.csv")
df

In [ ]:
df.info()

In [ ]:
df.columns
df[['MIN_TEMP_GROUND',
       'VIEW_RANGE', 'CLOUD',
       'WEATHER_CODE']].head(15)

In [ ]:
# mask with boolean indexing on nan values
df.isnull()

In [ ]:
df.isnull().any() # for columns

In [ ]:
df.isnull().any(axis=1)

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
# check if thera columns with all values as nan
df.isnull().all() 

In [ ]:
# the same for rows
df.isnull().all(axis=1) 
# to see only those rows with true
df.isnull().all(axis=1).any() 

In [ ]:
df.loc[df.isnull().all(axis=1) ]

In [ ]:
# the opposite of isnull
df.notnull().all()

In [ ]:
df['MIN_TEMP_GROUND'].head(15)

In [ ]:
every_6th_row = pd.Series(range(5, len(df), 6))

In [ ]:
df['MIN_TEMP_GROUND'].loc[every_6th_row] # select evey 6th row

In [ ]:
# to check that indeed all of the 6th rows have proper values
df['MIN_TEMP_GROUND'].loc[every_6th_row].notnull().all()

In [ ]:
# let's check that the values in the rest of the column (those that are not in the 6th row) are indeed nan
df['MIN_TEMP_GROUND'].drop(every_6th_row).isnull().all()

### Handling Missing Data

In [ ]:
df.info()

In [ ]:
# to remove a column
df.drop(columns='WEATHER_CODE', inplace=True)

In [ ]:
df

In [ ]:
# fill missing values
df['MIN_TEMP_GROUND'].fillna(0)

In [ ]:
df['MIN_TEMP_GROUND'].fillna(method='ffill')

In [ ]:
df['MIN_TEMP_GROUND'].fillna(method='bfill', inplace=True)

In [ ]:
# let's check if the values are indeed filled
df.isnull().any()
# 'MIN_TEMP_GROUND' shows False, meaning there are no missing values

In [ ]:
# let's see rows with missing data
df[df.isnull().any(axis=1)]

In [ ]:
# to check the dates on which the data is missing we can use loc
df.loc[df.isnull().any(axis=1), 'YYYYMMDD']
# the dates are repeating a lot, so we can use value_counts to see how many times each date appears

In [ ]:
df.loc[df.isnull().any(axis=1), 'YYYYMMDD'].value_counts()
# it means there are 20 rows for 20160821 with missing data

In [ ]:
df.info()

In [ ]:
# drop rows with na
nulls_dropped = df.dropna()
nulls_dropped.info()

In [ ]:
drop_thresh = df.dropna(thresh=7)
drop_thresh[drop_thresh.isnull().any(axis=1)]

In [ ]:
drop_thresh.fillna(0, inplace=True)
drop_thresh[drop_thresh.isnull().any(axis=1)]

In [ ]:
# select all rows with null values
rows_to_fill = df.isnull().any(axis=1)
df.loc[rows_to_fill]

In [ ]:
# fill with the mean of the column
nulls_filled = df.fillna(df.mean())
# check the filled rows. we use previously created rows_to_fill
nulls_filled.loc[rows_to_fill]

In [ ]:
# fill with mode insted of mean
df.fillna(df.mode().iloc[0])

In [ ]:
df.fillna(df.mode().iloc[0], inplace=True)

### Handling bad data other than nan


In [ ]:
athletes = pd.read_csv(r"E:/Downloads/_Pandas/Pandas Playbook. Manipulating Data (Reindert-Jan Ekker, 2018)/athletes.csv")
athletes.info()

In [ ]:
%matplotlib inline
athletes.plot.scatter(x='height', y='weight')

In [ ]:
from cProfile import label


heights = athletes['height']
heights.plot.box()

In [ ]:
q1 = heights.quantile(0.25)
q3 = heights.quantile(0.75)
iqr = q3 - q1

pmin = q1 - 1.5 * iqr
pmax = q3 + 1.5 * iqr

In [ ]:
nwh = heights.where(heights.between(pmin, pmax), heights.mean()) # boolean mask that selects
# true for values that are between pmin and pmax and puts the mean value for the rest
# IF nothing is specified for the false values, they are replaced with NaN!!
# heights.where(heights.between(pmin, pmax))

In [ ]:
# compare the original and the new values after the transformation
compare  = pd.DataFrame({'before': heights, 'after': nwh})
compare.head(15)

In [ ]:
compare  = pd.DataFrame({'before': heights, 'after': nwh})
compare.plot.box()
compare.describe()

In [ ]:
heights.where(heights.between(pmin, pmax), inplace=True)

In [ ]:
athletes.plot.scatter(x='height', y='weight')

### Handling duplicates

In [ ]:
# check if there are duplicate rows
athletes.duplicated().any()

In [ ]:
# select the duplicated rows
athletes[athletes.duplicated()]

In [ ]:
# remove the duplicated rows
athletes.drop_duplicates(inplace=True)

In [ ]:
athletes.duplicated().any()

In [ ]:
# just to explore the data and see unique values
athletes['nationality'].drop_duplicates() # returns cleaned df
# athletes['nationality'].unique() # returns numpy array

In [ ]:
athletes['nationality'].drop_duplicates().sort_values()

In [ ]:
# value counts
athletes['nationality'].value_counts()

In [ ]:
athletes['sex'].value_counts()

### Type Conversion

In [ ]:
athletes.info()

In [ ]:
athletes[['gold', 'silver', 'bronze']]

In [ ]:
# sum() on a string, just concatenates it
athletes['bronze'].sum()

In [ ]:
athletes['bronze'].astype(int)
# error - ValueError: invalid literal for int() with base 10: 'O'
# there is a row with string 'O' and not a zero. let's fix it

In [ ]:
# let's find all the rows with string 'O' in column 'bronze'
athletes[athletes['bronze'] == 'O']

In [ ]:
# let's now change the string 'O' to 0 for this specific row
athletes.loc[7521, ['gold', 'silver', 'bronze']] = 0

# and now, let's change the type of these columns to int 
athletes[['gold', 'silver', 'bronze']] = athletes[['gold', 'silver', 'bronze']].astype(int)
athletes[['gold', 'silver', 'bronze']].sum()


In [ ]:
athletes.info()

### Fixing Indexes

In [ ]:
athletes.head()

In [ ]:
athletes.index = athletes['id']

In [ ]:
athletes

In [ ]:

# setting an existing column as index
athletes.set_index('id', drop=True, inplace=True)
athletes.head()

In [ ]:
# renaming columns
athletes.rename(columns={'nationality': 'country', 'sport': 'discipline'}, inplace=True)
athletes.head()

In [ ]:
# renaming rows
athletes.rename(index={736041664: 000, 532037425: 1111}, inplace=True)
athletes.head()

In [ ]:
## reseting index
df = pd.read_csv(r"E:/Downloads/_Pandas/Pandas Playbook. Manipulating Data (Reindert-Jan Ekker, 2018)/weather_m4.csv")

In [ ]:
df.dropna(inplace=True)
df.info()

In [ ]:
df

In [ ]:
df.reset_index()

In [ ]:
df.reset_index(drop=True)

### Data Transformation

In [ ]:
import numpy as np
import pandas as pd
    

In [ ]:
df = pd.DataFrame(np.ones([5,4]), columns=['a', 'b', 'c', 'd'])
df

In [ ]:
df * 2

In [ ]:
# actually changing the original df
df *= 2
df

In [ ]:
# select a row and dibide it by 2
df.loc[1] /= 2

In [ ]:
# or a column
df['b'] -= 1
df

In [ ]:
 # new df
df2 = pd.DataFrame(np.ones([3,2]), columns=['d', 'e'], index=[2,4,5])
df2

In [ ]:
df + df2

In [ ]:
df.loc[2] * df2.loc[5]

In [ ]:
df - df.mean()


In [ ]:
df

In [ ]:
df - pd.Series({'a': 5, 'b': 5, 'e': 5, 'f': 5})

In [ ]:
# subtract mean of rows from each row
df.sub(df.mean(axis=1), axis=0)

### Function Application

In [ ]:
df = pd.DataFrame({'sin': np.arange(0, 5*np.pi, 0.01),
                   'cos':np.arange(0.5*np.pi, 5.5*np.pi, 0.01)})
df
#' apply sine function to the df
df = np.sin(df)
%matplotlib inline

df.plot()

In [ ]:
# custom function
# calculate the interquartile range
def iqr(col):
    return col.quantile(0.75) - col.quantile(0.25)

In [ ]:
# passing the custom function to the apply method
df.apply(iqr) # this way it's applied to columns

In [ ]:
# now to rows
df.apply(iqr, axis=1) # this way it's applied to rows

In [ ]:
def somefunc(x):
    return np.abs(x+.25)

In [ ]:
# let's apply the function to every cell in the df. We can't use apply as it works on rows and columns
df.applymap(somefunc) # important not to use () around the function name as it's the applymap that will trigger it


In [ ]:
df.applymap(somefunc).plot()

### Groupby

In [ ]:
athletes = pd.read_csv(r"E:/Downloads/_Pandas/Pandas Playbook. Manipulating Data (Reindert-Jan Ekker, 2018)/athletes_clean.csv")
athletes.info()

In [ ]:
g = athletes.groupby('nationality')

In [ ]:
g.sum()

In [ ]:
 g = athletes.groupby('nationality')[['gold', 'silver', 'bronze']]
 

In [ ]:
g.sum()

In [ ]:
g = athletes.groupby('sport')[['weight', 'height']]
g.mean()

In [ ]:
g = athletes.groupby(['sport', 'sex'])[['weight', 'height']]
g.mean()

In [ ]:
g = athletes.groupby(['sport', 'sex'])[['weight', 'height']]
# g.last()
# g.first()
g.max()

### Stack, unstack, pivot, pivot_table

In [ ]:
m = pd.read_csv(r"E:/Downloads/_Pandas/Pandas Playbook. Manipulating Data (Reindert-Jan Ekker, 2018)/monthly_data.csv")
m

In [ ]:
# change index
# m.set_index('YYYY', inplace=True)
m

In [ ]:
# to move values from all the 12 moths columns into a single column
m.stack().head(30)

In [ ]:
m.stack().sum()

In [ ]:
# unstack
w = athletes.groupby(['sport', 'sex'])['weight'].mean()
w

In [ ]:
# now we'll move it from all the weights being in one column to separate columns for female and male, since
# these are are the values from the internal group - sex
w.unstack()

In [ ]:
# pivot

p = pd.DataFrame({'id': [823905, 823905,
                         235897, 235897, 235897,
                         983422, 983422],
                  'item': ['prize', 'unit', 
                           'prize', 'unit', 'stock', 
                           'prize', 'stock'],
                  'value': [3.49, 'kg',
                            12.89, 'l', 50,
                            0.49, 4]})
p

In [ ]:
# pivot
p.pivot('id', 'item', 'value') 
# 'id' will be the column for each individual record. It will use different values from 'item' as different columlns
# and it will fill these new coumns from values from 'value'

In [ ]:
grades = pd.DataFrame([[6, 4, 5], [7, 8, 7], [6, 7, 9], [6, 5, 5], [5, 2, 7]], 
                       index = ['Mary', 'John', 'Ann', 'Pete', 'Laura'],
                       columns = ['test_1', 'test_2', 'test_3'])
grades.reset_index(inplace=True)
grades

In [ ]:
grades.melt()

In [ ]:
grades.melt(id_vars=['index']) # specify the index column

### Combining DataF

In [ ]:
grades = pd.DataFrame([[6, 4, 5], [7, 8, 7], [6, 7, 9], [6, 5, 5], [5, 2, 7]], 
                       index = ['Mary', 'John', 'Ann', 'Pete', 'Laura'],
                       columns = ['test_1', 'test_2', 'test_3'])
grades

In [ ]:
# add new column to an existing df
grades['test_4'] = pd.Series({'John': 5, 'Ann': 8, 'Pete': 9, 'Mary': 7, 'Laura': 10})
grades

In [ ]:
# now adding a new row
grades.loc['Bob'] = [2, 3, 4, 5]
grades

In [ ]:
# append
new_row = pd.Series({'test_1': 8, 'test_2': 9, 'test_3': 7, 'test_4': 8}, name='Kim')
grades.append(new_row)

In [ ]:
# addging a new column
grades['stud_nr'] = [113, 121, 123, 135, 139, 141]
# order colulmns as we need them
grades = grades[['stud_nr', 'test_1', 'test_2', 'test_3', 'test_4']]
grades

In [ ]:
other = pd.DataFrame([[139, 7, 7],
                       [123, 8, 6],
                       [142, 4, 5],
                       [113, 7, 9],
                       [155, 10, 9],
                       [121, 6, 4]], 
                       columns = ['stud_nr', 'exam1', 'exam2'])
other

In [ ]:
# combine two df with merge. it's like join in SQL
grades.merge(other) # by default it's an inner join

In [ ]:
grades.merge(other, how='left') # will show those students present in grades df

In [ ]:
grades.merge(other, how='right') # will show those students present in OTHER df

In [ ]:
grades.merge(other, how='outer') # will show those students present in both dfs

In [ ]:
pd.read_csv(r"E:\app_data\dropbox_13f_files\crspq.csv", nrows=100).head(50)

In [2]:

%%time
# concatenate different files with the same cik in their name into one file per cik

import os, glob
import numpy as np
import pandas as pd
from pathlib import Path
from fnmatch import fnmatch

processed_tables_parquet_till_2013 = r"E:\app_data\dropbox_13f_files\processed_tables\processed_tables_parquet_till_2013"
cik_parquet_till_2013 = r"E:\app_data\dropbox_13f_files\processed_tables\cik_parquet_till_2013"

# list all files in a folder and subfolders


def files_in_dir(dir, ext="*.parquet"):
    l = []
    for path, subdirs, files in os.walk(dir):
        for name in files:
            if fnmatch(name, ext):
                l.append(os.path.join(path, name))
    return l

parquet_list = files_in_dir(processed_tables_parquet_till_2013)


# extract a list of ciks from a list of file names 
def get_list_of_cik(parquet_list: list) -> list:
    cik_list = []
    for file in parquet_list:
        cik_list.append(file.split('\\')[-1].split('-')[0])
    return sorted(cik_list)

cik_list = get_list_of_cik(parquet_list)
cik_list = [int(x) for x in list(set(cik_list))]
# cik_list = [315066]

data_dir = Path(processed_tables_parquet_till_2013)
for cik in cik_list:
# cik = 9749
# temp_dir = r"E:\app_data"
# full_df = pd.concat(pd.read_parquet(f) for f in data_dir.glob(f"{cik}-*.parquet"))

    full_df = pd.concat(pd.read_parquet(f) for f in data_dir.glob(f"{cik}-*.parquet"))
    full_df = full_df[full_df.filedAsOfDate <= '2013-12-31']

    full_df['cusip'] = full_df['cusip9'].astype(str).str.upper()
    full_df['titleOfClass'] = full_df.get('titleOfClass',default=np.nan)
    full_df['nameOfIssuer'] = full_df.get('titleOfClass',default=np.nan)
    full_df['managerName'] = full_df.get('managerName',default=np.nan)
    full_df['putCall'] = full_df.get('putCall',default=np.nan)
    full_df['shrsOrPrnAmt'] = full_df.get('shrsOrPrnAmt',default=np.nan)
    full_df['sharesValue'] = full_df.get('sharesValue',default=np.nan)
    full_df['sharesValue'] = full_df['sharesValue'] * 1000
    
    full_df['sharesHeldAtEndOfQtr'] = full_df.get('sharesHeldAtEndOfQtr', default=np.nan)
    full_df['submissionType'] = full_df.get('submissionType',default=np.nan)
    full_df['quarter'] = full_df['periodOfReport'].dt.year.astype(str)+"Q"+full_df['periodOfReport'].dt.quarter.astype(str)
    full_df['updated_at'] = pd.Timestamp.now()
    full_df.drop(columns=["cusip8", "cusip9"], inplace=True)
    
    # if there are no rows in the df, then move on to another file
    if full_df.shape[0] == 0: continue
        
    full_df = (full_df.sort_values(by=['cusip', 'periodOfReport', 'filedAsOfDate', 'accessionNumber'], ascending=False))
    fdate = full_df.filedAsOfDate.dt.date.max()

    dtypes = {'cusip': str, 'nameOfIssuer': str , 'titleOfClass': str, 'periodOfReport': np.datetime64,
            'putCall': str, 'shrsOrPrnAmt': str, 'sharesValue': float, 'sharesHeldAtEndOfQtr': float, 
            'filedAsOfDate': np.datetime64, 'accessionNumber': str, 'cikManager' : int, 'managerName': str,
            'quarter': str, 'submissionType': str, 'updated_at': np.datetime64}
    
    # grouping by cusip and summing up shares and value
    attributes = {'accessionNumber': 'last', 'cikManager':'last', 'managerName':'last',
    'periodOfReport':'last', 'submissionType':'last', 'filedAsOfDate':'first', 'cusip':'last', 'nameOfIssuer': 'last',
    'titleOfClass' :'last', 'sharesValue':'last',  'sharesHeldAtEndOfQtr':"last", "shrsOrPrnAmt":"last",
    'putCall':'last', 'xml_flag':'last', 'updated_at':'last', 'quarter':'last'}

    full_df  = full_df.groupby(["cikManager","cusip", "quarter"], as_index=False).agg(attributes)

    full_df.astype(dtypes).to_parquet(f"{cik_parquet_till_2013}/{cik}-{fdate}.parquet", engine='pyarrow')
        


CPU times: total: 43min 31s
Wall time: 51min


In [4]:

%%time
from fnmatch import fnmatch

processed_tables_csv = r"E:\app_data\dropbox_13f_files\processed_tables\processed_tables"
my_filings_parquet = r"E:\app_data\sec_apps_data\speed_test\filings_13f_full\filings_parquet"

def files_in_dir_list(dir, ext="*.parquet"):
    l = []
    for path, subdirs, files in os.walk(dir):
        for name in files:
            if fnmatch(name, ext):
                l.append(os.path.join(path, name))
    return l

l = files_in_dir_list(my_filings_parquet)

CPU times: total: 1.38 s
Wall time: 1.38 s


In [109]:

%%time
from fnmatch import fnmatch

processed_tables_csv = r"E:\app_data\dropbox_13f_files\processed_tables\processed_tables"
my_filings_parquet = r"E:\app_data\sec_apps_data\speed_test\filings_13f_full\filings_parquet"
cik_parquet_till_2013 = r"E:\app_data\dropbox_13f_files\processed_tables\cik_parquet_till_2013"

def files_in_dir_dict(dir, ext="*.parquet"):
    d = {}
    for path, subdirs, files in os.walk(dir):
        for index, name in enumerate(files):
            if fnmatch(name, ext):

                    
                    # d[index] = os.path.join(path, name)
                d[index] = int(name.split("-")[-4].split("\\")[-1])
                # l.append(os.path.join(path, name))
    return d

l = files_in_dir_dict(cik_parquet_till_2013)


CPU times: total: 15.6 ms
Wall time: 15.6 ms


In [ ]:
# Getting a value and if doesn't exist assigning a default one
full_df['transaction_value'] = full_df.get('transaction_value',default=np.nan)

### .merge()

In [1]:
import pandas as pd
df1 = pd.DataFrame({'name': ['Fred', 'Suzy', 'Suzy', 'Bob'],
                    'pet': ['Dog', 'Dog', 'Cat', 'Fish']})

df2 = pd.DataFrame({'Name': ['Suzy', 'Suzy', 'Suzy', 'Fred', 'Joe', 'Joe'],
                    'Color ': ['Black', 'Blue', 'Red', 'Green', 'Yellow', 'Blue']})


In [2]:
display(df1), display(df2)

,name,pet
0,Fred,Dog
1,Suzy,Dog
2,Suzy,Cat
3,Bob,Fish


,Name,Color
0,Suzy,Black
1,Suzy,Blue
2,Suzy,Red
3,Fred,Green
4,Joe,Yellow
5,Joe,Blue


(None, None)

In [27]:
# be careful with names. Only because there was a white space after the column `name `, the code below didn't want to work
(df1.merge(df2.assign(name=df2.Name))) # by default it's an inner join

# df1.merge(df2, left_on='name', right_on='Name')

,name,pet,Name,Color
0,Fred,Dog,Fred,Green
1,Suzy,Dog,Suzy,Black
2,Suzy,Dog,Suzy,Blue
3,Suzy,Dog,Suzy,Red
4,Suzy,Cat,Suzy,Black
5,Suzy,Cat,Suzy,Blue
6,Suzy,Cat,Suzy,Red


In [5]:
(df1.merge(df2.assign(name=df2.Name), how='left')) # all value from df1 will be brought

,name,pet,Name,Color
0,Fred,Dog,Fred,Green
1,Suzy,Dog,Suzy,Black
2,Suzy,Dog,Suzy,Blue
3,Suzy,Dog,Suzy,Red
4,Suzy,Cat,Suzy,Black
5,Suzy,Cat,Suzy,Blue
6,Suzy,Cat,Suzy,Red
7,Bob,Fish,NaN,NaN


In [23]:
(df1.merge(df2.assign(name=df2.Name), how='right'))
    # .assign(pet = df1.pet.fillna("no_pet"),
    #        name = df1.name.replace("Suzy", 'No Suzy')))

,name,pet,Name,Color
0,Suzy,Dog,Suzy,Black
1,Suzy,Cat,Suzy,Black
2,Suzy,Dog,Suzy,Blue
3,Suzy,Cat,Suzy,Blue
4,Suzy,Dog,Suzy,Red
5,Suzy,Cat,Suzy,Red
6,Fred,Dog,Fred,Green
7,Joe,NaN,Joe,Yellow
8,Joe,NaN,Joe,Blue


In [7]:
df1

,name,pet
0,Fred,Dog
1,Suzy,Dog
2,Suzy,Cat
3,Bob,Fish


In [ ]:
# This code snippet is for when we are not sure if the column `transaction_value` exists at all. 
# If it doesn't, it will get automatically created with the default value
full_df['transaction_value'] = full_df.get('transaction_value',default=np.nan)

In [111]:
df1 = (df.merge(cusip_md_df[['cusip', 'quarter','ticker', 'cusip_description', 'nameofissuer', 'titleofclass']], on=['cusip', 'quarter'])
      .rename(columns={'ticker':'cusip_ticker'}))
